In [1]:
import pandas as pd
import json
import jsonlines 
from collections import defaultdict
import os
import sys
from src.common import *

In [2]:
root_directory = 'C:/Users/shri1/reasoning_about_actions/data/data_files/zero_shot_data'
files = [os.path.join(root_directory,file) for file in os.listdir(root_directory) if file.endswith('.jsonl')]
# files

In [3]:
new_df = pd.read_json(files[0],lines=True)

In [4]:
def calculate_question_stats(file):
    import pandas as pd
    df = pd.read_json(file, lines=True)
    dataset_name = file.split('\\')[-1].split('.')[0]
    result_dict = {}

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        category = row[SK_CATEGORY]
        plan_length = row[SK_PLAN_LENGTH]
        answer_type = row[OUT_OBJ_ANSWER]
        
        # If the category is not yet in the result_dict, initialize it
        if category not in result_dict:
            result_dict[category] = {}
        
        # If the plan length is not yet in the category, initialize it
        if f'plan_length_{plan_length}' not in result_dict[category]:
            result_dict[category][f'plan_length_{plan_length}'] = {'True_questions': 0, 'False_questions': 0, 'none_questions': 0,'free_form_questions': 0}
        
        # Increment the count based on answer type
        if answer_type == 'True':
            result_dict[category][f'plan_length_{plan_length}']['True_questions'] += 1
        elif answer_type == 'False':
            result_dict[category][f'plan_length_{plan_length}']['False_questions'] += 1
        elif answer_type == 'None':
            result_dict[category][f'plan_length_{plan_length}']['none_questions'] += 1
        else:
            result_dict[category][f'plan_length_{plan_length}']['free_form_questions'] += 1
    import pandas as pd

    # Initialize an empty list to hold dictionaries
    data_list = []

    # Loop over the result_dict and construct dictionaries
    for key, value in result_dict.items():
        for k, v in value.items():
            row_dict = {
                'plan_length': k,
                'question_category': key,
                'True_questions': v['True_questions'],
                'False_questions': v['False_questions'],
                'none_questions': v['none_questions'],
                'free_form_questions': v['free_form_questions']
            }
            data_list.append(row_dict)

    # Convert the list of dictionaries to a DataFrame
    analysis_df = pd.DataFrame(data_list)

    # Print the DataFrame
    analysis_df['total_questions'] = analysis_df['True_questions'] + analysis_df['False_questions'] + analysis_df['none_questions'] + analysis_df['free_form_questions']
    custom_order = ['plan_length_1', 'plan_length_5', 'plan_length_10', 'plan_length_15', 'plan_length_19']
    sorted_df = analysis_df.sort_values(by=['question_category', 'plan_length'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))
    # sorted_df    

    # Initialize a list to store the modified rows
    new_row = []
    total_true = 0
    total_false = 0
    total_none = 0
    total_free_form = 0
    total_questions = 0
    count = 0

    # Iterate over the DataFrame rows
    for index, row in sorted_df.iterrows():
        # Append the current row to the new_row list
        new_row.append(row)
        
        # Update totals for each column
        total_true += row['True_questions']
        total_false += row['False_questions']
        total_none += row['none_questions']
        total_free_form += row['free_form_questions']
        total_questions += row['total_questions']
        
        count += 1
        # Check if the current row is a multiple of 7 (after every 7 rows)
        if count % 7 == 0:
            # Create a dictionary for the totals
            totals_row = {'plan_length': row[0],
                'question_category': 'Total',
                'True_questions': total_true,
                'False_questions': total_false,
                'none_questions': total_none,
                'free_form_questions': total_free_form,
                'total_questions': total_questions
            }
            # Append the totals row to the new_row list
            new_row.append(totals_row)
            
            # Reset totals for the next 7 rows because we have 7 categories
            total_true = 0
            total_false = 0
            total_none = 0
            total_free_form = 0
            total_questions = 0

    # If the total number of rows is not a multiple of 7, append totals for the remaining rows
    if count % 7 != 0:
        totals_row = {'plan_length': row[0],
            'question_category': 'Total',
            'True_questions': total_true,
            'False_questions': total_false,
            'none_questions': total_none,
            'free_form_questions': total_free_form,
            'total_questions': total_questions
        }
        new_row.append(totals_row)

    dict_series = []
    for item in new_row:
        if isinstance(item, pd.Series):
            my_df = item.to_frame().T
        else:
            my_df = pd.DataFrame(item, index=[0])
        dict_series.append(my_df)       
        
    final_df = pd.concat(dict_series, ignore_index=True)
    # return final_df.to_csv(f'C:/Users/shri1/reasoning_about_actions/results/generated_questions_stats/{dataset_name}.csv', index=False)
    return final_df

    


In [5]:
final_df = calculate_question_stats(files[0])
final_df 

C:\Users\shri1\AppData\Local\Temp\ipykernel_19252\775521125.py:82: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  totals_row = {'plan_length': row[0],


,plan_length,question_category,True_questions,False_questions,none_questions,free_form_questions,total_questions
0,plan_length_1,object_tracking,1,1,0,2,4
1,plan_length_1,fluent_tracking,2,2,0,2,6
2,plan_length_1,state_tracking,2,0,0,2,4
3,plan_length_1,action_executability,1,1,1,2,5
4,plan_length_1,effects,1,1,0,2,4
5,plan_length_1,numerical_reasoning,2,2,0,6,10
6,plan_length_1,hallucination,2,3,1,2,8
7,plan_length_1,Total,11,10,2,18,41
8,plan_length_5,object_tracking,0,2,0,2,4
9,plan_length_5,fluent_tracking,4,0,0,2,6


In [6]:
import pandas as pd
import os

agg_df = pd.DataFrame()

csv_dir = r'C:\Users\shri1\reasoning_about_actions\results\generated_questions_stats'

for filename in os.listdir(csv_dir):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(csv_dir, filename))
        
        # Select only the columns of interest
        df_selected = df[['True_questions', 'False_questions', 'none_questions', 'free_form_questions', 'total_questions']]
        
        # Add the selected columns to the aggregated DataFrame
        agg_df = agg_df.add(df_selected, fill_value=0)

# agg_df['total'] = agg_df.sum(axis=1)



In [7]:
selcted_columns = final_df[['plan_length','question_category']]

In [8]:
agg_df.insert(0, 'plan_length', selcted_columns['plan_length'])
agg_df.insert(1, 'question_category', selcted_columns['question_category'])

In [10]:
agg_df.drop(columns=['total'], inplace=True)

In [12]:
agg_df.to_csv(f'C:/Users/shri1/reasoning_about_actions/results/generated_questions_stats/aggregated_values.csv', index=False)